In [57]:
import numpy as np
import pandas as pd

In [58]:
# Set fileName, and run all.
# Upload appropriate Dataset
# logs.csv will contain the required data

fileName = 'btcusdt_1h.csv'

In [59]:
df = pd.read_csv('/content/'+fileName)

#Plotter

In [60]:
#@title Initial Values
signal_column = 'signals'
initialCash = 1000
cash = initialCash
priceColumn = 'close'
investmentAmount=1000
startingCapital = initialCash
commision=0.15

#Strategies

In [61]:
strategyNumber=0

##ESMA

In [62]:
length = 30
multiplier = 3
atrLength = 14
exp = True
def ESMA(data,exp, period, column="close"):
  if exp == True:
    return data[column].ewm(span=period, adjust=False).mean()
  else:
    return data[column].rolling(window=period).mean()

def ESMASignals(data,stratNumber):
  data["ma"] = ESMA(data,exp,length)
  # rangema = ATR LOGIC #### TODO ####
  data['High-Low'] = data['high'] - data['low']
  data['High-PrevClose'] = np.abs(data['high'] - data['close'].shift(1))
  data['Low-PrevClose'] = np.abs(data['low'] - data['close'].shift(1))
  data['TrueRange'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
  data["rangema"] = data['TrueRange'].rolling(window=atrLength).mean()
  data["upper"] = data["ma"] + data["rangema"]*multiplier
  data["lower"] = data["ma"] - data["rangema"] *multiplier
  data["crossUpper"] = 0
  data["crossLower"] = 0
  data["bprice"] = 0
  data["sprice"] = 0
  data["BCond"] = 0
  data["SCond"] = 0  # Same as cross upper and cross lower(unless there are nan values, none in our case)
  data["CancelBCond"] = 0
  data["CancelSCond"] = 0
  for i in range(1,len(data)):
    data.loc[i,"crossUpper"] = int(data.loc[i-1, "close"]< data.loc[i-1, "upper"] and data.loc[i,"close"]>data.loc[i,"upper"])
    data.loc[i,"crossLower"] = int(data.loc[i-1, "close"] > data.loc[i-1, "lower"] and data.loc[i,"close"]<data.loc[i,"lower"])
    data.loc[i,"bprice"] = data.loc[i,"high"] + data.loc[:i,"high"].min() if data.loc[i,"crossUpper"] == 1 else data.loc[i-1,"bprice"]
    data.loc[i,"sprice"] = data.loc[i,"low"] - data.loc[:i,"low"].min() if data.loc[i,"crossLower"] == 1 else data.loc[i-1,"bprice"]
    data.loc[i,"BCond"] = data.loc[i,"close"]>data.loc[i,"upper"]
    data.loc[i,"SCond"] = data.loc[i,"close"]<data.loc[i,"lower"]
    data.loc[i,"CancelBCond"] = data.loc[i,"BCond"] and (data.loc[i,"close"]<data.loc[i,"ma"] or data.loc[i,"high"]>=data.loc[i,"bprice"])
    data.loc[i, "CancelSCond"] = data.loc[i,"SCond"] and (data.loc[i,"close"]>data.loc[i,"ma"] or data.loc[i,"low"]<=data.loc[i,"sprice"])
  position = 0
  column = 'signals'+str(stratNumber)
  data[column] = 0
  for i in range(20,len(data)):
    if data.loc[i,"BCond"] == 1:
      if position != 1:
        data.loc[i,column] = 1
        position +=1
    elif data.loc[i,"SCond"]==1:
      if position != -1:
        data.loc[i,column] = -1
        position -= 1
    elif data.loc[i,"CancelBCond"] == 1 and position == 1:
      data.loc[i,column] = -1
      position -=1
    elif data.loc[i,"CancelSCond"] == 1 and position == -1:
      data.loc[i,column] = 1
      position +=1
    else:
      data.loc[i,column] = 0

In [63]:
ESMASignals(df,strategyNumber)
strategyNumber+=1
df

,datetime,open,high,low,close,volume,ma,High-Low,High-PrevClose,Low-PrevClose,...,lower,crossUpper,crossLower,bprice,sprice,BCond,SCond,CancelBCond,CancelSCond,signals0
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,13529.010000,315.64,NaN,NaN,...,NaN,0,0,0.00,0.00,0,0,0,0,0
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,13507.980968,440.51,66.88,373.63,...,NaN,0,0,0.00,0.00,False,False,False,False,0
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,13496.509938,218.43,215.37,3.06,...,NaN,0,0,0.00,0.00,False,False,False,False,0
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,13490.930587,321.27,281.09,40.18,...,NaN,0,0,0.00,0.00,False,False,False,False,0
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,13498.032484,301.14,213.26,87.88,...,NaN,0,0,0.00,0.00,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,42079.267611,403.07,123.21,279.86,...,40491.937611,0,0,54484.76,54484.76,False,False,False,False,0
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,42116.682604,179.95,23.31,156.64,...,40535.849747,0,0,54484.76,54484.76,False,False,False,False,0
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,42155.163081,178.73,116.94,61.79,...,40685.527367,0,0,54484.76,54484.76,False,False,False,False,0
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,42192.203528,252.31,173.15,79.16,...,40733.839242,0,0,54484.76,54484.76,False,False,False,False,0


##CMF

In [64]:
def cmf(df, stratNumber, thresh=0):
  df_hour = df.copy()
  df_hour["datetime"] = pd.to_datetime(df_hour["datetime"])
  df_hour.set_index("datetime", inplace = True)

  df_hour.shape

  # Resampling to daily  frequency
  df_day = pd.DataFrame()
  df_day['open'] = df_hour.resample("D").open.first()
  df_day['close'] = df_hour.resample("D").close.last()
  df_day['high']=df_hour.resample("D").high.max()
  df_day['low'] = df_hour.resample("D").low.min()
  df_day['volume'] = df_hour.resample("D").volume.sum()
  df_day.reset_index(inplace=True)
  # print(df_day)

  df_hour.reset_index(inplace=True)
  signals = np.zeros(df_day.shape[0])
  MFV = df_day.volume*(2*df_day.close - df_day.low - df_day.high)/(df_day.high - df_day.low)
  res = MFV.rolling(20).sum()/df_day.volume.rolling(20).sum()
  pos = 0
  for i in range(signals.shape[0]-1):
      if res[i] > thresh and pos != 1:
          signals[i+1] = 1
          pos += 1
      elif res[i] < -thresh and pos != -1:
          signals[i+1] = -1
          pos -= 1
  df_day['signals'] = signals

  signals1 = np.zeros(df.shape[0])
  j = 0
  for i in range(df.shape[0]-1):
    if df_hour.datetime.iloc[i+1].day != df_hour.datetime.iloc[i].day:
      signals1[i] = df_day.signals[j]
      j+=1
  df['signals'+str(stratNumber)] = signals1
  return df

In [65]:
df = cmf(df,strategyNumber)
strategyNumber+=1
df

,datetime,open,high,low,close,volume,ma,High-Low,High-PrevClose,Low-PrevClose,...,crossUpper,crossLower,bprice,sprice,BCond,SCond,CancelBCond,CancelSCond,signals0,signals1
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,13529.010000,315.64,NaN,NaN,...,0,0,0.00,0.00,0,0,0,0,0,0.0
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,13507.980968,440.51,66.88,373.63,...,0,0,0.00,0.00,False,False,False,False,0,0.0
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,13496.509938,218.43,215.37,3.06,...,0,0,0.00,0.00,False,False,False,False,0,0.0
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,13490.930587,321.27,281.09,40.18,...,0,0,0.00,0.00,False,False,False,False,0,0.0
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,13498.032484,301.14,213.26,87.88,...,0,0,0.00,0.00,False,False,False,False,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,42079.267611,403.07,123.21,279.86,...,0,0,54484.76,54484.76,False,False,False,False,0,0.0
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,42116.682604,179.95,23.31,156.64,...,0,0,54484.76,54484.76,False,False,False,False,0,0.0
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,42155.163081,178.73,116.94,61.79,...,0,0,54484.76,54484.76,False,False,False,False,0,0.0
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,42192.203528,252.31,173.15,79.16,...,0,0,54484.76,54484.76,False,False,False,False,0,0.0


##VVAP

In [66]:
def VWAP(data,stratNumber):
  vwap_window = 1000
  column = 'signals'+str(stratNumber)
  data["vwap"] = data["close"]
  data["cumvolprice"] = 0
  data["cumvol"]=0
  for i in range(vwap_window, len(data)):
    df2 = data[i-vwap_window:i].copy(deep=True)
    # print(data)
    # print(df["volume"].cumsum())
    data.loc[i,"cumvol"] = (df2["volume"].cumsum())[i-1]
    data.loc[i,"cumvolprice"] = ((df2['volume'] * (df2['high'] + df2['low'] + df2['close'] ) /3).cumsum())[i-1]
    data.loc[i,"vwap"] = data.loc[i,"cumvolprice"] / data.loc[i,"cumvol"]

  position = 0
  data[column] = 0
  for i in range(vwap_window,len(data)):
    if data.loc[i,"vwap"] < data.loc[i,"close"] :
      if position != 1:
        data.loc[i,column] = 1
        position +=1
    elif data.loc[i,"vwap"] > data.loc[i,"close"]:
      if position != -1:
        data.loc[i,column] = -1
        position -= 1
    else:
      data.loc[i,column] = 0
  return data

In [67]:
df = VWAP(df,strategyNumber)
strategyNumber+=1
df

,datetime,open,high,low,close,volume,ma,High-Low,High-PrevClose,Low-PrevClose,...,BCond,SCond,CancelBCond,CancelSCond,signals0,signals1,vwap,cumvolprice,cumvol,signals2
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,13529.010000,315.64,NaN,NaN,...,0,0,0,0,0,0.0,13529.010000,0.000000e+00,0.000000e+00,0
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,13507.980968,440.51,66.88,373.63,...,False,False,False,False,0,0.0,13203.060000,0.000000e+00,0.000000e+00,0
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,13496.509938,218.43,215.37,3.06,...,False,False,False,False,0,0.0,13330.180000,0.000000e+00,0.000000e+00,0
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,13490.930587,321.27,281.09,40.18,...,False,False,False,False,0,0.0,13410.030000,0.000000e+00,0.000000e+00,0
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,13498.032484,301.14,213.26,87.88,...,False,False,False,False,0,0.0,13601.010000,0.000000e+00,0.000000e+00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35203,2022-01-12 01:30:00,42972.04,43095.26,42692.19,42800.38,1219.601780,42079.267611,403.07,123.21,279.86,...,False,False,False,False,0,0.0,47968.680711,7.746669e+10,1.614943e+06,0
35204,2022-01-12 02:30:00,42797.62,42823.69,42643.74,42659.20,702.103800,42116.682604,179.95,23.31,156.64,...,False,False,False,False,0,0.0,47958.023435,7.745059e+10,1.614966e+06,0
35205,2022-01-12 03:30:00,42664.71,42776.14,42597.41,42713.13,561.859930,42155.163081,178.73,116.94,61.79,...,False,False,False,False,0,0.0,47949.704125,7.741844e+10,1.614576e+06,0
35206,2022-01-12 04:30:00,42713.12,42886.28,42633.97,42729.29,681.142010,42192.203528,252.31,173.15,79.16,...,False,False,False,False,0,0.0,47942.416987,7.738641e+10,1.614153e+06,0


#COMBINER

In [68]:
######################################

In [69]:
#COMBINER CODE
signal_column = 'signals'
windowPNLs = []
windowWeights = []

In [70]:
def getCompleteInfo(df,initialCash,position=0,entryPrice=0,exitPrice=0): #assumes signal represents entry/exit, PNL= (Exit Price−Entry Price) ×  Investment amount/Entry price, returns final portfolio value
  global signal_column, priceColumn, investmentAmount, commision
  portfolio = []
  #position = 0 #####
  cash = initialCash
  multipleBuys = False
  multipleSells = False
  #entryPrice = 0
  #exitPrice=0
  for index,row in df.iterrows():
    signal = row[signal_column]
    currPrice = row[priceColumn]
    if position==0:
      portfolio.append(cash)
      if signal==1:
        position=1
        entryPrice = currPrice
      elif signal==-1:
        position=-1
        exitPrice = currPrice
    elif position==1:
      portfolio.append(cash-investmentAmount+investmentAmount/entryPrice*currPrice)
      if signal==-1:
        change = investmentAmount/entryPrice*(currPrice-entryPrice)
        cash+=change
        cash-=commision/100*investmentAmount
        position=0
      elif signal==1:
        multipleBuys = True
    elif position==-1:
      portfolio.append(cash+investmentAmount-investmentAmount/exitPrice*currPrice)
      if signal==1:
        change = investmentAmount/exitPrice*(exitPrice-currPrice)
        cash+=change
        cash-=commision/100*investmentAmount
        position=0
      elif signal==-1:
        multipleSells = True
  if multipleBuys:
    print("Your signals attempt to hold more than 1 position")
  if multipleSells:
    print("Your signals attempt to sell more than 1 position")
  return portfolio[len(portfolio)-1]-portfolio[0]

In [71]:
#@title Main Utility Functions
def InitialiseCumSums(k):
  lst= []
  for i in range(k):
    lst.append(0)
  return lst

def InitialiseWeights(k):
  lst = []
  for i in range(k):
    lst.append(1/k)
  return lst

def GetWindow(idx,window):
  lst = []
  for i in range(idx,idx+window):
    lst.append(i)
  return lst

def GetWindowDF(df, idx, window):
  window = min(len(df)-idx,window)
  return df.iloc[GetWindow(idx,window)]

def UpdateCumSums(df,startingIndex,window,cumSums):
  global signal_column
  for w in range(window):
    for k in range(len(cumSums)):
      cumSums[k]+= df.iloc[startingIndex+w][signal_column+str(k)]
  return cumSums

def UpdateCumSumsAtIndex(df,index,cumSums,indices):
  global signal_column
  for k in range(len(cumSums)):
    cumSums[k]+=df.iloc[index][signal_column+str(indices[k])]
  return cumSums

def GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices):
  global signal_column
  PNLs = []
  minProfit = float('inf')
  prevSignalColumn = signal_column
  for i in range(numOfStrategies):
    signal_column = signal_column+str(i)
    PNLs.append(getCompleteInfo(GetWindowDF(df,startingIndex,window),currentCash,cumSums[i],entryPrices[i],exitPrices[i]))
    signal_column = prevSignalColumn
    minProfit = min(minProfit,PNLs[i])
  signal_column = prevSignalColumn
  return PNLs,minProfit

def GetPNLWeights(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices):
  global windowPNLs
  profitList,minProfit = GetPNLs(df,startingIndex,window,numOfStrategies,currentCash,cumSums,entryPrices,exitPrices)
  windowPNLs.append(profitList.copy())
  sum = 0
  for i in range(numOfStrategies):
    profitList[i]-=minProfit
    sum+=profitList[i]
  if sum==0:
    for i in range(numOfStrategies):
      profitList[i] = 1/numOfStrategies
  else:
    for i in range(numOfStrategies):
      profitList[i] = profitList[i]/sum
  return profitList
  #return [1,0,0,0,0]

def GetPosition(df,index,weights):
  position = 0
  for i in range(len(weights)):
    position+= weights[i]*df.iloc[index][signal_column+str(i)]
  if position>0:
    return 1
  if position<0:
    return -1
  else:
    return 0

def GetPositionByCumSums(cumSums,weights):
  position = 0
  for k in range(len(cumSums)):
    position+=weights[k]*cumSums[k]
  if position>0:
    return 1
  elif position<0:
    return -1
  else:
    return 0

def GetPositionByCumSumsThresh(cumSums,weights,thresh=0):
  position = 0
  for k in range(len(cumSums)):
    position+=weights[k]*cumSums[k]
  if position>thresh:
    return 1
  elif position<-thresh:
    return -1
  else:
    return 0

def Sign(x):
  if x>0:
    return 1
  elif x<0:
    return -1
  else:
    return 0

def GetPositionUsingMajorityCumSums(cumSums,m):
  sumOfPositions = 0
  for k in range(len(cumSums)):
    sumOfPositions+=cumSums[k]
  if abs(sumOfPositions)>=2*m-len(cumSums):
    if  sumOfPositions>0:
      return 1
    else:
      return -1
  else:
    return 0


def UpdatePricesAtIndex(df,index,cumSums,entryPrices,exitPrices):
  global priceColumn,signal_column
  price = df.iloc[index][priceColumn]
  for k in range(len(cumSums)):
    if cumSums[k]==1 and df.iloc[index][signal_column+str(k)]==1:
      entryPrices[k] = price
    elif cumSums[k]==-1 and df.iloc[index][signal_column+str(k)]==-1:
      exitPrices[k] = price
  return entryPrices,exitPrices


In [72]:
def dummyStrat():
  return 0

In [73]:
#@title Window Weight Strategy Threshold
#THOUGHTS:
#Each strategy runs for window iterations, then combiner uses signals from both strategies to get new set of signals

#Bookkeeping  for stratgies - entryPrice, exitPrice, cumSum
def CombineUsingWeightedWindowsThresh(df, strategyList, window, threshold, indices = []): #Assume that function takes l,r values to populate, ith strategy populates column signal+'i'
  global signal_column, priceColumn, investmentAmount, startingCapital,windowWeights
  if len(indices)==0:
    for i in range(len(strategyList)):
      indices.append(i)
  currentCash = startingCapital
  cumSums = InitialiseCumSums(len(strategyList))
  entryPrices = InitialiseCumSums(len(strategyList))
  exitPrices = InitialiseCumSums(len(strategyList))
  weights = InitialiseWeights(len(strategyList))
  signals = []
  position = 0

  for i in range(window):
    #cumSums = UpdateCumSumsAtIndex(df,i,cumSums)
    #entryPrices,exitPrices = UpdatePricesAtIndex(df,i,cumSums,entryPrices,exitPrices)
    signals.append(0)
  for i in range(window,len(df),window):

    weights = GetPNLWeights(df,i-window,window,len(strategyList),currentCash,cumSums,entryPrices,exitPrices)
    windowWeights.append(weights.copy())
    for j in range(i-window,min(i,len(df))):
      cumSums = UpdateCumSumsAtIndex(df,j,cumSums,indices)
      entryPrices,exitPrices = UpdatePricesAtIndex(df,j,cumSums,entryPrices,exitPrices)
    cumSumsCopy = cumSums.copy()
    for w in range(min(i+window,len(df))-i):
      #cumSums = UpdateCumSumsAtIndex(df,i+w,cumSums)
      #entryPrices,exitPrices = UpdatePricesAtIndex(df,i+w,cumSums,entryPrices,exitPrices)
      newPosition = GetPositionByCumSumsThresh(cumSumsCopy,weights,threshold)
      cumSumsCopy = UpdateCumSumsAtIndex(df,i+w,cumSumsCopy,indices)
      signal = Sign(newPosition-position) #if position = 1, and newPos = -1, signal=-1. i.e. position != newPosition
      signals.append(signal)
      currentCash += -1*signal*df.iloc[i+w][priceColumn]
      position += signal
    #i+=window
  df[signal_column] = signals
  df['Index'] = range(0,len(df))
  df = df[['Index','datetime','open','high','low','close','volume','signals']]
  df.to_csv('/content/logs.csv',encoding='utf-8',index=False)

In [74]:
CombineUsingWeightedWindowsThresh(df, [dummyStrat,dummyStrat,dummyStrat],700,0.4)